In [ ]:
# # pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# # You can check the column names from the readme file
# #Reading users file:

# users = pd.read_csv(r'C:\Users\Yash\Documents\0_ML\0_HW\hw2\netflix\TrainingRatings.txt', names=cols, sep='|', encoding='latin-1')

# cols2 = ['MovieID' ,'YearOfRelease' ,'Title']
# items = pd.read_csv(r'C:\Users\Yash\Documents\0_ML\0_HW\hw2\netflix\movie_titles.txt', names=cols2, sep='|',encoding='latin-1')

In [ ]:
n_users = training.UserID.unique().shape[0]
n_items = training.MovieID.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

In [ ]:
train_data_matrix = np.zeros((n_users, n_items))
for line in training.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

In [ ]:
# df_movie_features.replace(0, np.nan, inplace=True)
# df_movie_features.head()

In [ ]:
# final_movies = df_movie_features.fillna(df_movie_features.mean(axis=0))
# final_movies
# final_user = df_movie_features.apply(lambda row: row.fillna(row.mean()), axis=1)
# final_user

In [ ]:
# user similarity on replacing NAN by user avg
from sklearn.metrics.pairwise import cosine_similarity

b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

In [ ]:
# user similarity on replacing NAN by user avg
from sklearn.metrics.pairwise import cosine_similarity

b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

In [ ]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movies)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movies.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

In [ ]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
# top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,10)
sim_user_30_u.head()

In [ ]:
# for line in training[:3].itertuples():
#     print(line)
#     print(line[1], line[2], line[3])

In [ ]:
# train_data_matrix = np.zeros((n_users.shape[0]-1, n_items.shape[0]-1))
# for line in training.itertuples():
#     train_data_matrix[line[1]-1, line[2]-1] = line[3]

# test_data_matrix = np.zeros((n_users, n_items))
# for line in testing.itertuples():
#     test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [ ]:
item_prediction = predict(df_movie_features, cos_df, type='user')

In [ ]:
# corrMatrix = df_movie_features.corr(method='pearson')
# corrMatrix.head(100)

In [ ]:
most_sim_users = sorted(list(enumerate(cos_df[8])), key=lambda x: x[1], reverse=True)
most_sim_users = most_sim_users[1:11]
sim_users = [x[0] for x in most_sim_users]
print(sim_users)

In [ ]:
def func(ele):
    return ele[1]

def predict(userId, movieId):
    dict_with_rating = {}
    lis = []
    for idx, rating in enumerate(df_movie_features.iloc[movieId - 1]):
        if(rating > 0):
            temp = (idx,rating)
            dict_with_rating[idx] = rating
            lis.append(temp)
    top_k = []
    for i,sim in enumerate(cos_df.iloc[userId - 1]):
        for j,k in lis:
            if(j == i):
                top_k.append([i,sim])
    top_sorted_sim = sorted(top_k, key=cos_df.iloc[userId], reverse=True)
    top_5 = top_sorted_sim[:5]
    sum_ = 0
    sim_sum = 0
    for top in top_5:
        sum_ += dict_with_rating[top[0]]*top[1]
        sim_sum += top[1]
    return sum_ / sim_sum

In [ ]:
user_id = training.UserID
movie_id = training.MovieID 
rating_s = training.Rating

ui_train, ui_test, mi_train, mi_test, r_train, r_test = train_test_split(user_id, movie_id, rating_s, test_size=0.2, random_state = 2)

In [ ]:
print(ui_test.shape)
print(mi_test.shape)
print(r_test.shape)

In [ ]:
em_list = []
for j in range(28978):
    predicted_rats = predict(ui_test.iloc[j], mi_test.iloc[j])
    rating_Q = r_test.iloc[j]
    half_rmse = (predicted_rats - rating_Q)**2
    em_list.append(half_rmse)
rmse = math.sqrt(sum(em_list)/len(em_list))
print("RMSE:",rmse)

In [ ]:
def person_corelation(person1,person2):
    both_rated = {}
    for item in training[person1]:
        if item in training[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)
    if number_of_ratings == 0:
        return 0

    person1_preferences_sum = sum([training[person1][item] for item in both_rated])
    person2_preferences_sum = sum([training[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(training[person1][item], 2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(training[person2][item], 2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([training[person1][item] * training[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (
    person1_preferences_sum * person2_preferences_sum / number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum, 2) / number_of_ratings) * (
    person2_square_preferences_sum - pow(person2_preferences_sum, 2) / number_of_ratings))
    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

In [ ]:
def weight_factor(x, y):
    ''' 
    Weight factor implies relationship between user x and user y
    Also know as similarity between user x and user y
    We are using Pearson correlation coefficient here. 
    '''    
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

n = 1000
active_user_id = training.iloc[n, 0]

active_user = training[training['UserID'] == active_user_id]
active_user_rating = active_user.iloc[:, 2:]
active_user_rating

# saving active user ratings into 1 d list
active_user_rating_list = active_user_rating.values.ravel()
# finding similarity between active user and all its neighbours among complete rating users
similarity = np.array([(training.iloc[i, 0],\
             weight_factor(active_user_rating_list, training.iloc[i, 2:]))\
             for i in range(training.shape[0])])

In [ ]:
from numpy import sqrt 
def calcSim (user, userOther, data):
    sum_xy = 0
    sum_x2 = 0
    sum_y2 = 0
    n = 0
    rating1 = data[user]
    rating2 = data[userOther]

    for key in rating1.all()>0:
        if key in rating2>0:
            n += 1
            sum_xy += rating1[key]*rating2[key]


    if n == 0:
        return 0
    else:

        for value in rating1[:]:
            sum_x2 += value**2
        for value in rating2[:]:
            sum_y2 += value**2

        denominator = sqrt(sum_x2) * sqrt(sum_y2)

        if denominator == 0:
            return 0
        else:
            return sum_xy/denominator

calcSim(1,2,train_data_matrix)

In [ ]:
#This function finds k similar users given the user_id and ratings matrix M
#Note that the similarities are same as obtained via using pairwise_distances
global k,metric
k=4
metric='correlation'
from sklearn.neighbors import NearestNeighbors

def findksimilarusers(user_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)
    txt = '{0} most similar users for User {1}:\n'
    print(txt.format(k,UserID))
    distances, indices = model_knn.kneighbors(ratings[user_id-1, :].reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
            
    return similarities,indices

In [ ]:
similarities,indices = findksimilarusers(7,train_data_matrix, metric='correlation')

In [ ]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings[user_id-1,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            ratings_diff = ratings[indices.flatten()[i],item_id-1]-np.mean(ratings[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    return prediction

In [ ]:
p = predict_userbased(7,8,train_data_matrix);